### Boltzmann Machine for movie recommendation system

In [53]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [54]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [55]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

### Calculating number of users and movies

In [56]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

### Matrix Conversion

In [57]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

### Converting matrix to Torch Tensors

In [58]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

### Converting ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [59]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

### Creating the architecture of the RBM

In [60]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh) # bias 2D vector with 1 as batch dimension and nh corresponding to bias
    self.b = torch.randn(1, nv)
  def sample_h(self, x): # This function activates hidden nodes as per some probability
    wx = torch.mm(x, self.W.t()) # vector of weights multiplied by x ==> x is vector of visible neurons
    activation = wx + self.a.expand_as(wx) # expand_as makes sure bias is applied to each line of mini_batch
    p_h_given_v  = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk): # This is contrastive divergence algorithm implementation
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
rbm = RBM(nv, nh)

### Training the RBM

In [61]:
def TrainModel(nb_epoch, batch_size):
  for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    counter = 0. # . added to make counter as float for loss normalization
    for id_user in range(0, nb_users - batch_size, batch_size):
      vk = training_set[id_user: id_user + batch_size]
      v0 = training_set[id_user: id_user + batch_size] # Original input to be compared with predicted results for calculating loss
      ph0,_ = rbm.sample_h(v0) # ,_ only returns first element of bernoulli sampling
      for k in range(10):
        _,hk = rbm.sample_h(vk) # _, means return second element
        _, vk = rbm.sample_v(hk)
        vk[v0<0] = v0[v0<0]
      phk,_ = rbm.sample_h(vk)
      rbm.train(v0, vk, ph0, phk)
      train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
      counter += 1.
  print('Epoch: '+str(epoch)+' Loss: '+str(train_loss/counter))  

In [63]:
TrainModel(100, 32)

Epoch: 100 Loss: tensor(0.2463)


In [67]:
test_loss = 0
counter = 0.
for id_user in range(nb_users):
  v = training_set[id_user: id_user + 1]
  vt = test_set[id_user: id_user + 1]
  # Blind walk begins to go to prediction
  if len(vt[vt >= 0]) > 0:
    _,h = rbm.sample_h(v) 
    _, v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
    counter += 1.
print('test loss: '+str(test_loss/counter))

test loss: tensor(0.2460)
